# DBSETUP.ipynb

Welcome to the database setup jupyter notebook!
Assuming one has all of the necessary packages, this notebook will serve as a click-and-go way to set up the Automated Flake Detection database!

You should have come from INSTALL.md; if you haven't completed the steps in there, then this notebook probably won't work!

## LOGIN
Before we start, enter your root login info below. You should probably not commit this to the database seeing as it's, you know, your private info. You will need to run this block at least once before doing anything else in this notebook. It's also a good time to check and make sure you have the mysql-connector-python package, but that should be handled already if you've already followed the installation instructions.

In [22]:
from mysql.connector import Error, connect
import os
import json

username = "root" # your username
pw = "" # replace with your password as a string 

# loads up password from vscode/settings.json; likely not the case for you unless you've already set it up
try:
    FILE_DIR = os.path.dirname(os.path.abspath(''))
    with open(os.path.join(FILE_DIR, "Automated-Flake-Detection", ".vscode", "settings.json")) as f:
        pw = json.load(f)['sqltools.connections'][0]['password']
        print("Password found in .vscode/settings.json!")
except:
    if pw == "":
        print("No password found under .vscode/settings.json; you should type your password in manually.")
    else:
        print("Password manually changed; continuing with your password.")
password =  pw 

Password found in .vscode/settings.json!


## Database Creation
Creates a new database named AFD_db under localhost, or your computer. Future iterations will hopefully include an option to just connect to a central database.

In [23]:
# NEEDS TO BE RUN AFTER LOGIN BLOCK IS POPULATED + RAN
try:
    with connect(
        host = "localhost",
        user = username,
        password = password,
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute("CREATE DATABASE AFD_db")
            connection.commit()
except Error as e:
    print(e)

1007 (HY000): Can't create database 'afd_db'; database exists


## Table Creation
Creates the Flake and Chip tables in the AFD_db database.

In [11]:
# NEEDS TO BE RUN AFTER LOGIN BLOCK IS POPULATED + RAN
create_chip_table = """
    CREATE TABLE Chips(
        chip_id INT PRIMARY KEY AUTO_INCREMENT,
        material VARCHAR(255) NOT NULL,
        size INT NOT NULL,
        img VARCHAR(255) 
)"""

create_flake_table = """
    CREATE TABLE Flakes(
        chip_id INT NOT NULL,
        flake_id INT,
        FOREIGN KEY (chip_id) REFERENCES Chips(chip_id),
        PRIMARY KEY (chip_id, flake_id),
        thickness VARCHAR(7),
        size INT,
        center_x INT,
        center_y INT,
        confidence FLOAT,
        low_mag VARCHAR(255),
        med_mag VARCHAR(255),
        high_mag VARCHAR(255)
)
"""

try:
    with connect(
        host = "localhost",
        user = username,
        password = password,
        database = "AFD_db",
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(create_chip_table)
            cursor.execute(create_flake_table)
            connection.commit()
except Error as e:
    print(e)

1050 (42S01): Table 'chips' already exists


## All done, return to INSTALL.md!